Given function is:$$f(x)=400x_1^2+0.004x_2^4
$$
then the hassian matrix of the function will be:
$$\nabla^2f(x)=\begin{bmatrix}
800 &0\\
0& 0.048x_2^2
\end{bmatrix}$$

In [ ]:
import numpy as np 

In [ ]:
#method to find the condition number of any square matrix: : Complete the code
def find_condition_number(A):
  assert type(A) is np.ndarray
  assert A.shape[0] == A.shape[1]
  k=max(np.linalg.eig(A)[0])/min(np.linalg.eig(A)[0])
  return k

In [ ]:
#method to find Hessian matrix: Complete the code
def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  return np.array([[800,0],[0,0.048*x[1]**2]])

In [ ]:
#Now we will define a Python function which will compute and return the objective function value 
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  return (400*x[0]**2 + 0.004*x[1]**4)

In [ ]:
#Now we will define a Python function which will compute and return the gradient value as a numpy array 
def evalg(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the gradient value
  #compute the gradient value and return it 
  return np.array([800*x[0],0.016*x[1]**3])

In [ ]:
#The method defines a way to construct D_k matrix used in scaling the gradient in the modified gradient descent scheme
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k
  d_k = np.array([[1/evalh(x)[0][0], 0],[0, 1/evalh(x)[1][1]]])
  return d_k

In [ ]:
#The method defines a way to construct D_k matrix used in scaling the gradient in the modified gradient descent scheme
def compute_D_k_nondiagonal(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k_1 for non diagonal D
  if np.linalg.det(evalh(x))==0:
    raise ValueError('Determinant is 0')
  d_k = np.linalg.inv(evalh(x))
  return d_k

In [ ]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 

  alpha = alpha_start
  #implement the backtracking line search
  p= -gradf
  while(evalf(x+alpha*p) > evalf(x) + gamma*alpha*np.dot(gradf.transpose(),p)):
    alpha = rho*alpha


  #print('final step length:',alpha)
  return alpha

In [ ]:
def compute_steplength_backtracking_scaled_direction(x, gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(direction) is np.ndarray and len(direction) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  while evalf(x + alpha*direction) > ( evalf(x) +  (gamma*alpha*np.dot(gradf,direction)) ):
    alpha = rho*alpha
  return alpha

In [ ]:
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH_SCALING = 4

In [ ]:
#complete the code for gradient descent to find the minimizer
def find_minimizer_gd(start_x, tol, line_search_type,*args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  
  #Complete the code   
  A = np.array([[1500, 2],[2,1]])
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    # print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)
    condition_num_arr = []
  k = 0
  # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    condition_num_arr.append(find_condition_number(evalh(x)))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k, condition_num_arr

In [ ]:
#complete the code for gradient descent with scaling to find the minimizer

def find_minimizer_gdscaling(start_x, tol, line_search_type,*args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  A = np.array([[1500, 2],[2,1]])
  x = start_x
  g_x = evalg(x)
  d_k = compute_D_k(x)
  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH_SCALING):
    alpha = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    direction = -np.matmul(d_k,g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH_SCALING:
      step_length = compute_steplength_backtracking_scaled_direction(x,g_x,direction,alpha,rho,gamma)
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.matmul(np.multiply(step_length,g_x),d_k)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x ,k

In [ ]:
#complete the code for gradient descent with scaling to find the minimizer
import scipy
from scipy.linalg import sqrtm
def find_minimizer_gdscaling_non_diagonal(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)

  condition_number_scaling = []

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH_SCALING):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]

  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    d_k = compute_D_k_nondiagonal(x)

    if line_search_type == BACKTRACKING_LINE_SEARCH_SCALING:
      
      direction = np.matmul(-d_k,g_x)
      
      p = scipy.linalg.sqrtm(d_k)
      hessn = np.matmul(np.matmul(p,evalh(x)),p)
      condition_number_scaling.append(find_condition_number(hessn))
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x,direction, alpha_start, rho, gamma) #call the new function you wrote to compute the steplength 
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.0
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here  
    x = np.subtract(x, np.multiply(step_length,np.matmul(d_k, g_x))) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

  return x, k, condition_number_scaling

## **Answer 3:**

In [ ]:
my_tol = 1e-6
alpha = 1.0
rho = 0.5
gamma = 0.5
my_start_x = np.array([2.0,2.0])

In [ ]:
#Compute gradient descent with scaling and with backtracking line search 

print("Newtons method with scaling and constant step length")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k, condition2 = find_minimizer_gdscaling_non_diagonal(my_start_x, my_tol, CONSTANT_STEP_LENGTH)
iterations2 = range(k)
print("Optimizer: ", x_opt_bls)
print('Minimum Value: ', evalf(x_opt_bls))
print("Number of iterations: ", k)

Newtons method with scaling and constant step length
Optimizer:  [0.         0.03468306]
Minimum Value:  5.788014517642641e-09
Number of iterations:  10


In [ ]:
#Compute gradient descent with scaling and with backtracking line search 

print("Newton's method with Backtracking Line Search With scaling:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k, condition2 = find_minimizer_gdscaling_non_diagonal(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH_SCALING,alpha, rho, gamma)
iterations2 = range(k)
print("Optimizer: ", x_opt_bls)
print('Minimum Value: ', evalf(x_opt_bls))
print("Number of iterations: ", k)

Newton's method with Backtracking Line Search With scaling:
Optimizer:  [0.         0.03468306]
Minimum Value:  5.788014517642641e-09
Number of iterations:  10


### **Observations:** 
***Using Newtons method with scaling and constant step length:*** \
Optimizer:  [0.         0.03468306] \
Minimum Value:  5.788014517642641e-09 \
Number of iterations:  10 \

***Using Newton's Method with Backtracking Line Search With scaling:*** \
Optimizer:  [0.         0.03468306] \
Minimum Value:  5.788014517642641e-09 \
Number of iterations:  10 \


Here we can observe that the solution in both the cases is coming out to be exactly the same. The number of iterations taken in both the cases are equal to 10 and the optimizer and the functional values are also the same.

## **Answer 4:**

In [ ]:
tol= 1e-6

In [ ]:
#Compute gradient descent with scaling and with backtracking line search 

print("Newtons method without scaling and step length by backtracking:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k, condition2 = find_minimizer_gd(my_start_x, tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
iterations2 = range(k)
print("Optimizer: ", x_opt_bls)
print('Minimum Value: ', evalf(x_opt_bls))
print("Number of iterations: ", k)

Newtons method without scaling and step length by backtracking:
Optimizer:  [3.53148711e-10 3.91377499e-02]
Minimum Value:  9.385197325158433e-09
Number of iterations:  5707240


In [ ]:
#Compute gradient descent with scaling and with backtracking line search 

print("Gradient Descent method with scaling using a diagonal matrix and step length by backtracking:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k= find_minimizer_gdscaling(my_start_x, tol, BACKTRACKING_LINE_SEARCH_SCALING, alpha, rho, gamma)
iterations2 = range(k)
print("Optimizer: ", x_opt_bls)
print('Minimum Value: ', evalf(x_opt_bls))
print("Number of iterations: ", k)

Gradient Descent method with scaling using a diagonal matrix and step length by backtracking:
Optimizer:  [0.         0.03967984]
Minimum Value:  9.916069790614023e-09
Number of iterations:  3803


### **Observations:**

**Note:** Here we have used the tolerance as 1e-6 as the time taken to solve the problems with tolerance as 1e-9 was taking a lot of time.


***Using Gradient Descent method without scaling and step length by backtracking:*** \
Optimizer:  [3.53148711e-10 3.91377499e-02] \
Minimum Value:  9.385197325158433e-09 \
Number of iterations:  5707240 \

***Using Gradient Descent method with scaling using a diagonal matrix and step length by backtracking:*** \
Optimizer:  [0.         0.03967984] \
Minimum Value:  9.916069790614023e-09 \
Number of iterations:  3803 \

Here we can see that the number of iterations taken by gradient descent without scaling is 5707240 which is very large as compared to that in Gradient Descent with scaling(3803), even though the minimum value in both the cases are of the same order. Also in GD with scaling, the $x_1$ variable is achieving its real value of 0 while its not the case in GD without scaling.